
# Notebook 06  Micro-Expression Cleaning & Dataset Comparison
Last polished on: 2025-10-15

**Notebook Purpose**  
This notebook prepares the micro-expression datasets (SMIC and CASME II) for symbolic audit and multimodal fusion. Tasks include:

- Loading and inspecting SMIC and CASME II raw metadata
- Cleaning emotion labels, removing bad samples, and flagging ambiguity
- Aligning emotion categories with DAIC-WOZ taxonomy
- Preparing `.parquet` outputs for use in fusion and Z3 logic audits

This step is foundational for building symbolic rules on frame-level emotion masking, dissociation, and ambiguous affect patterns (Notebook 07).


In [ ]:
# =============================================================================
# 06.0  Initialization & Path Setup
# -----------------------------------------------------------------------------
# - Sets up all paths for data and output locations
# - Confirms directories exist or creates them
# - Confirms notebook is loaded correctly
# =============================================================================

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Define root paths (adjusted for notebook use) ----------------------------
ROOT = Path().resolve().parent  # One level up from /notebooks/
DATA_DIR = ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
CHECKS_DIR = ROOT / "outputs" / "checks"
VIS_DIR = ROOT / "outputs" / "visuals"

# --- Create folders if missing ------------------------------------------------
CHECKS_DIR.mkdir(parents=True, exist_ok=True)
VIS_DIR.mkdir(parents=True, exist_ok=True)

# --- Startup confirmation -----------------------------------------------------
print("✅ Notebook 06 initialized successfully")
print(f"📂 Root directory:       {ROOT}")
print(f"📂 Raw data directory:   {RAW_DIR}")
print(f"📂 Processed directory:  {PROCESSED_DIR}")
print(f"📂 Outputs > Checks:     {CHECKS_DIR}")
print(f"📂 Outputs > Visuals:    {VIS_DIR}")




---
## 6.1 | CASME II Cleaning & Structuring

**Source:** `CASME2-coding-20140508.xlsx`  
**Output:** `casme2_cleaned.parquet`  

**Goal:**

- Load CASME II label + timing metadata  
- Clean and standardize columns (onset, offset, AUs, etc.)  
- Normalize emotion labels (lowercase)  
- Output tidy .parquet for fusion



In [ ]:
print("✅ RAW_DIR:", RAW_DIR)
print("🛣️ Full path:", RAW_DIR / "CASME2" / "CASME2-coding-20140508.xlsx")



In [ ]:
# =============================================================================
# 6.1  Load CASME II Metadata File
# =============================================================================
# Goal:
#   - Read Excel spreadsheet containing CASME II label + timing metadata
#   - Confirm file loads successfully and preview structure
#   - Check for column names, shape, and class label distribution
# =============================================================================

# --- Define CASME II spreadsheet path ----------------------------------------
casme_meta_path = RAW_DIR / "CASME2" / "CASME2-coding-20140508.xlsx"

# --- Load the spreadsheet safely ---------------------------------------------
try:
    casme_df = pd.read_excel(casme_meta_path)
    print(f"✅ Loaded CASME II metadata: {casme_df.shape}")
except FileNotFoundError:
    print(f"❌ File not found: {casme_meta_path}")
    casme_df = None

# --- Preview structure if loaded correctly -----------------------------------
if casme_df is not None:
    # Display first few rows
    display(casme_df.head(3))
    
    # Show column types and null counts
    display(casme_df.info())

    # View emotion label distribution (if column exists)
    if 'Estimated Emotion' in casme_df.columns:
        print("Unique emotion labels:")
        print(casme_df['Estimated Emotion'].value_counts())



In [ ]:
# =============================================================================
# 6.1.2 Clean & Standardize CASME II Metadata
# -----------------------------------------------------------------------------
# - Drops empty or irrelevant columns (e.g., Unnamed)
# - Normalizes emotion labels for consistency
# - Renames key columns for model fusion (onset, offset, duration)
# - Handles any formatting issues with ApexFrame, Action Units, etc.
# =============================================================================

# --- Drop junk or empty columns ----------------------------------------------
drop_cols = [col for col in casme_df.columns if "Unnamed" in col]
casme_df.drop(columns=drop_cols, inplace=True)
print(f"🧹 Dropped columns: {drop_cols}")

# --- Rename for fusion alignment ---------------------------------------------
casme_df.rename(columns={
    "OnsetFrame": "Onset",
    "OffsetFrame": "Offset",
    "ApexFrame": "Peak",
    "Subject": "SubjectID",
    "Estimated Emotion": "Emotion",
    "Action Units": "ActionUnits"
}, inplace=True)

# --- Normalize emotion labels ------------------------------------------------
casme_df["Emotion"] = casme_df["Emotion"].str.strip().str.lower()

# --- Normalize ActionUnits if not already string -----------------------------
casme_df["ActionUnits"] = casme_df["ActionUnits"].astype(str)

# --- Handle ApexFrame (Peak) column safely -----------------------------------
casme_df["Peak"] = pd.to_numeric(casme_df["Peak"], errors="coerce")

# --- Add derived column: Duration (Offset - Onset) ---------------------------
casme_df["Duration"] = casme_df["Offset"] - casme_df["Onset"]

# --- Confirm changes ---------------------------------------------------------
display(casme_df.head())
display(casme_df.info())
print("✅ Cleaned Emotion label distribution:")
print(casme_df["Emotion"].value_counts())


In [ ]:
# =============================================================================
# 6.1.3 Save Cleaned CASME II Metadata (Safe Version)
# -----------------------------------------------------------------------------
# Converts mixed-type columns to string so PyArrow can serialize them.
# Replaces invalid numeric entries with NaN and stores as text-safe Parquet.
# =============================================================================

CASME_CLEANED_PATH = CHECKS_DIR / "casme2_cleaned.parquet"

# --- Optional: Force all columns to string for safety ------------------------
casme_df = casme_df.astype(str)

# --- Save to Parquet ---------------------------------------------------------
casme_df.to_parquet(CASME_CLEANED_PATH, index=False)
print(f"✅ Saved CASME II cleaned metadata → {CASME_CLEANED_PATH}")


In [ ]:
# =============================================================================
# 6.1.4 🕷️ Spider Check (Sanity check):
# -----------------------------------------------------------------------------
# # confirm file was saved and list folder contents
# =============================================================================

if CASME_CLEANED_PATH.exists():
    print(f"✅ File exists at expected path: {CASME_CLEANED_PATH}")
    print("\n📂 Contents of checks folder:")
    for f in CHECKS_DIR.glob("*"):
        print(" -", f.name)
else:
    print(f"❌ File not found at: {CASME_CLEANED_PATH}")



---
## 6.2 | SMIC Cleaning & Structuring

**Source:** `SMIC_VIS_E`, `SMIC_NIR_E`, and `SMIC_HS_E` Excel annotations  
**Output:** `smic_cleaned.parquet`

**Goal:**

- Load all three SMIC subsets (VIS, NIR, HS)  
- Clean and unify structures  
- Normalize emotion labels and create shared frame fields  
- Save consolidated .parquet for fusion



In [ ]:
# =============================================================================
# 🗂️  File Discovery — Locate All .xlsx Files in RAW_DIR
# -----------------------------------------------------------------------------
# Goal:
#   - Recursively search through all subdirectories in the raw data folder
#   - Identify and print full paths of all Excel (.xlsx) files
#   - Helps confirm visibility of annotation spreadsheets and debug path issues
# =============================================================================

from pathlib import Path

for p in RAW_DIR.rglob("*.xlsx"):
    print("📄 Found:", p)


In [ ]:
# =============================================================================
# 6.2.1  Load SMIC VIS-E Metadata
# =============================================================================
# Goal:
#   - Read Excel spreadsheet containing visual modality emotion annotations
#   - Tag rows with modality = "VIS" for fusion use later
#   - Confirm successful load and inspect structure and class labels
# =============================================================================

# --- Define spreadsheet path for SMIC VIS-E modality -------------------------
vis_path = RAW_DIR / "SMIC" / "SMIC-E_raw image" / "VIS_long" / "SMIC_VIS_E_annotation.xlsx"

# --- Load the spreadsheet safely ---------------------------------------------
try:
    vis_df = pd.read_excel(vis_path)
    vis_df['Modality'] = "VIS"  # Add modality tag
    print(f"✅ Loaded VIS-E metadata: {vis_df.shape}")
except FileNotFoundError:
    print(f"❌ VIS-E metadata not found at: {vis_path}")
    vis_df = None

# --- Preview structure if loaded correctly -----------------------------------
if vis_df is not None:
    # Show first few rows
    display(vis_df.head())
    
    # Show column types and null counts
    display(vis_df.info())

    # View emotion label distribution
    print("✅ VIS-E label distribution:")
    print(vis_df['Emotion'].value_counts())



In [ ]:
# =============================================================================
# 6.2.2  Load SMIC NIR-E Metadata
# =============================================================================
# Goal:
#   - Read Excel spreadsheet containing NIR modality annotations
#   - Add modality tag for alignment in fusion step
#   - Confirm successful load and inspect label structure
# =============================================================================

# --- Define spreadsheet path for SMIC NIR-E modality -------------------------
nir_path = RAW_DIR / "SMIC" / "SMIC-E_raw image" / "NIR_long" / "SMIC-NIR-E_annotation.xlsx"

# --- Load the spreadsheet safely ---------------------------------------------
try:
    nir_df = pd.read_excel(nir_path)
    nir_df['Modality'] = "NIR"  # Add modality tag
    print(f"✅ Loaded NIR-E metadata: {nir_df.shape}")
except FileNotFoundError:
    print(f"❌ NIR-E metadata not found at: {nir_path}")
    nir_df = None

# --- Preview structure if loaded correctly -----------------------------------
if nir_df is not None:
    # Show first few rows
    display(nir_df.head())
    
    # Show column types and null counts
    display(nir_df.info())

    # View emotion label distribution
    print("✅ NIR-E label distribution:")
    print(nir_df['Emotion'].value_counts())



In [ ]:
# =============================================================================
# 6.2.2  Load SMIC NIR-E Metadata
# =============================================================================
# Goal:
#   - Read Excel spreadsheet containing NIR modality annotations
#   - Add modality tag for alignment in fusion step
#   - Confirm successful load and inspect label structure
# =============================================================================

# --- Define spreadsheet path for SMIC NIR-E modality -------------------------
nir_path = RAW_DIR / "SMIC" / "SMIC-E_raw image" / "NIR_long" / "SMIC-NIR-E_annotation.xlsx"

# --- Load the spreadsheet safely ---------------------------------------------
try:
    nir_df = pd.read_excel(nir_path)
    nir_df['Modality'] = "NIR"  # Add modality tag
    print(f"✅ Loaded NIR-E metadata: {nir_df.shape}")
except FileNotFoundError:
    print(f"❌ NIR-E metadata not found at: {nir_path}")
    nir_df = None

# --- Preview structure if loaded correctly -----------------------------------
if nir_df is not None:
    # Show first few rows
    display(nir_df.head())
    
    # Show column types and null counts
    display(nir_df.info())

    # View emotion label distribution
    print("✅ NIR-E label distribution:")
    print(nir_df['Emotion'].value_counts())



In [ ]:
# =============================================================================
# 6.2.3  Load SMIC HS-E Metadata
# =============================================================================
# Goal:
#   - Load high-speed modality Excel file for SMIC
#   - Add modality tag for downstream processing
#   - Validate structure and review emotion label distribution
# =============================================================================

# --- Define spreadsheet path for SMIC HS-E modality --------------------------
hs_path = RAW_DIR / "SMIC" / "SMIC-E_raw image" / "HS_long" / "SMIC-HS-E_annotation_2019.xlsx"

# --- Load the spreadsheet safely ---------------------------------------------
try:
    hs_df = pd.read_excel(hs_path)
    hs_df['Modality'] = "HS"  # Add modality tag
    print(f"✅ Loaded HS-E metadata: {hs_df.shape}")
except FileNotFoundError:
    print(f"❌ HS-E metadata not found at: {hs_path}")
    hs_df = None

# --- Preview structure if loaded correctly -----------------------------------
if hs_df is not None:
    # Show first few rows
    display(hs_df.head())
    
    # Show column types and null counts
    display(hs_df.info())

    # View emotion label distribution
    print("✅ HS-E label distribution:")
    print(hs_df['Emotion'].value_counts())



In [ ]:
# =============================================================================
# 6.2.4 Clean & Concatenate SMIC Modalities
# -----------------------------------------------------------------------------
# Goal:
#   - Standardize column structure across VIS, NIR, and HS SMIC subsets
#   - Drop irrelevant or noisy columns (e.g., Unnamed, extra onset/offsets)
#   - Normalize emotion labels and frame column names
#   - Concatenate into one unified DataFrame: smic_df
# =============================================================================

import pandas as pd

# --- Define common columns to keep across all subsets ------------------------
COMMON_COLS = ["Subject", "Filename", "OnsetF", "OffsetF", "FirstF", "TotalVL", "Emotion", "Modality"]

# --- Select and copy only needed columns for each subset ---------------------
vis_clean = vis_df[COMMON_COLS].copy()
nir_clean = nir_df[COMMON_COLS].copy()
hs_clean  = hs_df[COMMON_COLS].copy()

# --- Rename columns to unified format ----------------------------------------
rename_cols = {
    "OnsetF": "Onset",
    "OffsetF": "Offset",
    "FirstF": "First",
    "TotalVL": "Duration"
}

vis_clean.rename(columns=rename_cols, inplace=True)
nir_clean.rename(columns=rename_cols, inplace=True)
hs_clean.rename(columns=rename_cols, inplace=True)

# --- Normalize emotion labels ------------------------------------------------
for df in [vis_clean, nir_clean, hs_clean]:
    df["Emotion"] = df["Emotion"].str.strip().str.lower()

# --- Add source tag for traceability -----------------------------------------
for df in [vis_clean, nir_clean, hs_clean]:
    df["SourceDataset"] = "SMIC"

# --- Concatenate all three subsets -------------------------------------------
smic_df = pd.concat([vis_clean, nir_clean, hs_clean], ignore_index=True)
print(f"✅ Unified SMIC dataset created: {smic_df.shape}")

# --- Final structure check ---------------------------------------------------
display(smic_df.head(3))
display(smic_df.info())
print("✅ Emotion label distribution:")
print(smic_df["Emotion"].value_counts())


In [ ]:
# =============================================================================
# 6.2.5 💾 Save Cleaned SMIC Metadata (Safe Version)
# -----------------------------------------------------------------------------
# - Converts all columns to string so PyArrow can serialize mixed types
# - Stores result as a safe Parquet file for later fusion in Section 6.3
# - Location: outputs/checks/smic_cleaned.parquet
# =============================================================================

SMIC_CLEANED_PATH = CHECKS_DIR / "smic_cleaned.parquet"

# --- Optional: Force all columns to string (safe across inconsistent types) --
smic_df = smic_df.astype(str)

# --- Save to Parquet ---------------------------------------------------------
smic_df.to_parquet(SMIC_CLEANED_PATH, index=False)
print(f"✅ Saved SMIC cleaned metadata → {SMIC_CLEANED_PATH}")

# --- 🕷️ Spider Check (Sanity check): confirm file was saved and list folder contents ----------
if SMIC_CLEANED_PATH.exists():
    print(f"📂 Contents of checks folder:")
    for f in CHECKS_DIR.glob("*"):
        print(" -", f.name)
else:
    print(f"❌ File not found at: {SMIC_CLEANED_PATH}")


---
## 6.3 | Load Cleaned Metadata for Fusion

Purpose:
 - Load the cleaned CASME II and SMIC metadata parquet files
 - Preview each dataset side by side before structural comparison
 - Ensure no data corruption and verify readiness for alignment


 **Input files**:

| Dataset    | Path                                         | Shape     |
|------------|----------------------------------------------|-----------|
| CASME II   | `outputs/checks/casme2_cleaned.parquet`      | (255, 8)  |
| SMIC       | `outputs/checks/smic_cleaned.parquet`        | (305, 9)  |


 **Note**: DAIC-WOZ is not used in this fusion. This is purely a facial microexpression metadata merge.



In [ ]:
# =============================================================================
# 6.3.1 Load Cleaned Metadata for Fusion
# =============================================================================
# Purpose:
#   - Load the cleaned CASME II and SMIC metadata parquet files
#   - Preview each dataset side by side before structural comparison
#   - Ensure no data corruption and verify readiness for alignment
# =============================================================================

# --- Load CASME II cleaned data ----------------------------------------------
casme_path = CHECKS_DIR / "casme2_cleaned.parquet"
casme_df   = pd.read_parquet(casme_path)
print(f"✅ CASME II loaded: {casme_df.shape}")

# --- Load SMIC cleaned data --------------------------------------------------
smic_path = CHECKS_DIR / "smic_cleaned.parquet"
smic_df   = pd.read_parquet(smic_path)
print(f"✅ SMIC loaded: {smic_df.shape}")

# --- Preview structure and content -------------------------------------------
display(casme_df.head(3))
display(smic_df.head(3))

# --- Optional: Schema alignment diagnostic -----------------------------------
print("🔷 CASME II columns:", casme_df.columns.tolist())
print("🔷 SMIC columns:    ", smic_df.columns.tolist())


---
### 6.3.2 — Align CASME II and SMIC Metadata for Fusion

This section ensures both datasets have identical schemas so we can concatenate them safely for multimodal emotion modeling.

What will be done:
- Rename SMIC’s Subject to match CASME’s SubjectID
- Add missing columns to CASME II: Modality, SourceDataset
- Fill missing Peak in SMIC with NaN
- Reorder columns consistently across both datasets
  >Final Schema (Target Order): 
      >["SubjectID", "Filename", "Onset", "Peak", "Offset", "ActionUnits", "Emotion", "Duration", "Modality", "SourceDataset"]


In [ ]:
# =============================================================================
# 6.3.2 Align CASME II and SMIC Schemas for Fusion
# -----------------------------------------------------------------------------
# Ensures consistent column names, types, and order between datasets
# to allow seamless concatenation and downstream modeling.
# =============================================================================

import numpy as np

# --- Rename SMIC's Subject → SubjectID ---------------------------------------
smic_df.rename(columns={"Subject": "SubjectID"}, inplace=True)

# --- Add missing columns to CASME II -----------------------------------------
casme_df["Modality"] = "CASME2"
casme_df["SourceDataset"] = "CASME2"

# --- Add missing column to SMIC ----------------------------------------------
smic_df["Peak"] = np.nan
smic_df["ActionUnits"] = np.nan  # If not already present — safe placeholder

# --- Reorder columns to match target schema ----------------------------------
COL_ORDER = ["SubjectID", "Filename", "Onset", "Peak", "Offset", "ActionUnits",
             "Emotion", "Duration", "Modality", "SourceDataset"]

casme_df = casme_df[COL_ORDER]
smic_df = smic_df[COL_ORDER]

# --- Concatenate both datasets -----------------------------------------------
fusion_df = pd.concat([casme_df, smic_df], ignore_index=True)

# --- Quick structure check ---------------------------------------------------
print(f"✅ Fused dataset shape: {fusion_df.shape}")
display(fusion_df.head(3))
display(fusion_df.info())
print("✅ Fused emotion distribution:")
print(fusion_df["Emotion"].value_counts())


In [ ]:
# =============================================================================
# 6.3.3 💾 Save Fused Microexpression Metadata (Safe Version)
# -----------------------------------------------------------------------------
# Purpose:
#   - Persist combined SMIC + CASME II metadata as .parquet
#   - Ensures full reproducibility for Notebook 07 (Modeling & Analysis)
#   - Converts columns to string (where needed) for Arrow safety
# =============================================================================

FUSED_META_PATH = CHECKS_DIR / "fused_microexpression_metadata.parquet"

# --- Optional: force to string if you want absolute safety -------------------
# fusion_df = fusion_df.astype(str)

# --- Save to .parquet --------------------------------------------------------
fusion_df.to_parquet(FUSED_META_PATH, index=False)
print(f"✅ Saved fused metadata → {FUSED_META_PATH}")

# --- 🕷️Spider Check: Confirm file exists -----------------------------------------------------
if FUSED_META_PATH.exists():
    print("📂 Contents of checks folder:")
    for f in CHECKS_DIR.glob("*"):
        print(" -", f.name)
else:
    print("❌ Save failed — file not found at expected path.")


---
#  Notebook 06 Complete: Microexpression Cleaning & Fusion Summary

---

## Purpose

This notebook performs structured loading, cleaning, and fusion of two multimodal microexpression datasets:

- **CASME II**: Chinese Academy of Sciences Micro-Expression II
- **SMIC**: Spontaneous Micro-Expression Corpus (VIS-E, NIR-E, HS-E modalities)

These datasets are prepped for downstream emotion modeling, rule extraction, and semantic verification (Notebook 07+).

---

## Outputs

Cleaned and fused `.parquet` files saved to `outputs/checks/`:

| File | Description |
|------|-------------|
| `casme2_cleaned.parquet` | Cleaned CASME II metadata |
| `smic_cleaned.parquet` | Unified SMIC metadata across all 3 modalities |
| `fused_microexpression_metadata.parquet` | Final aligned table (CASME II + SMIC) for modeling |

---

## Final Dataset Summary

-  **Total Records**: 560  
-  **Emotion Classes**: `['negative', 'positive', 'surprise', 'disgust', 'repression', 'happiness', 'sadness', 'fear', 'others']`  
-  **Modality Tags**: VIS, NIR, HS, CASME II  
-  **Schema Harmonized**: `['SubjectID', 'Filename', 'Onset', 'Peak', 'Offset', 'ActionUnits', 'Emotion', 'Duration', 'Modality', 'SourceDataset']`

---

## Next Steps

Proceed to **Notebook 07**:

>  Frame-level expansion, feature extraction, and semantic affect labeling  
>  Prepare fusion dataset for rule mining and multimodal emotion modeling

---

##  Acknowledgments

This work reflects trauma-informed alignment goals. Each entry represents more than facial frames — it’s a step toward systems that **see what others overlook.**

